In [1]:
import requests
import os
import pandas as pd
from datetime import date, datetime, timedelta
import time
from io import StringIO
idag = date.today().isoformat()[0:10]
tokenCanvas = os.environ['tokenCanvas']

headers = {'Authorization': f'Bearer {tokenCanvas}',}
params = {'per_page': 100,}

# Initialisering

Først må eg lese fila over alle som er aktive lærarar i Canvas, og sjekke om dei er medlem i eit testemne. Dette vert berre gjort ein gang, og resultatet vert lagra i ei fil. 


In [11]:
data = pd.read_csv("users_filtered.csv")
data['brukarnamn'] = data['login_id'].apply(lambda x: x.split('@')[0])
eksisterande = data[data['brukarnamn'].str.isalpha()]
eksisterande.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1471 entries, 182 to 18680
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         1471 non-null   object 
 1   integration_id  0 non-null      float64
 2   login_id        1471 non-null   object 
 3   first_name      1471 non-null   object 
 4   last_name       1471 non-null   object 
 5   short_name      0 non-null      float64
 6   email           1471 non-null   object 
 7   status          1471 non-null   object 
 8   brukarnamn      1471 non-null   object 
dtypes: float64(2), object(7)
memory usage: 114.9+ KB


Den neste kodesnutten lager ei liste over alle lærarar, og kva testemne dei eventuelt er med i. Ikkje køyr denne fleire ganger!

In [ ]:
# for person in data['login_id']:
#     print(f"{person} | ", end="")
#     try:
#         params = {'per_page': 100}
#         url = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term={person}"
#         respons = requests.get(url, headers=headers, params=params)
#         user_id = respons.json()[0]['id']
#         name = respons.json()[0]['name']
#         print(f"{user_id} | {name} | ", end="")
#         emneliste = []
#         params = {'per_page': 100, 'state': ['unpublished']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         params = {'per_page': 100, 'state': ['available']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         for e in emneliste: 
#             if 'Testemne' in e:
#                 print(f"{e}, ", end=" ")
#         print("")
#     except:
#         print("Ikkje lærar.") 

## Opprette testemne for alle lærarar som ikkje har det.

Eg må gå gjennom fila frå forrige blokk og plukke ut alle som har "Lag nytt emne" i Kontrollfelt:


In [ ]:
# data = pd.read_excel("250506 oversiktsfil.xlsx")
# # data.head()

In [ ]:
# lag_testemne = data[data['Kontrollfelt'] == 'Lag nytt emne']
# lag_testemne.head()


Den neste kodesnutten går gjennom alle eg har funnet og oppretter nye testemne. Ikkje køyr denne fleire ganger!

In [ ]:
# for n in range(11,len(lag_testemne)):
#     i = lag_testemne.iloc[n]
#     params = {
#         'course[name]': f"Testemne for {i['Namn']}",
#         'course[sis_course_id]': f"hvl250506-e{n}-5-akv",
#         'course[course_code]': f"Testemne for {i['Namn']}",
#         'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne.\nTing du lager her kan du hente inn i andre emne til bruk i undervising.",
#         'course[default_view]': 'syllabus', 
#     }
#     url = f"https://hvl.instructure.com/api/v1/accounts/9044/courses"
#     response = requests.post(url, headers=headers, params=params)
#     course_id = response.json()['id']
#     params = {
#         'enrollment[user_id]': i['user_id'],
#         'enrollment[type]': 'TeacherEnrollment',
#         'enrollment[workflow_state]': 'active',
#     }
#     url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/enrollments"
#     response = requests.post(url, headers=headers, params=params)
#     print(f"{i['user_id']}: {response.status_code}")

# Neste steg: sjekke nye lærarar kvar natt

No må eg lage meg ei kontrolliste med kven som har testemne. Kvar natt sjekker eg så importen av lærarar, og ser om det er nokon der ssom ikkje er på lista over eksisterande lærarar. Desse skal i så fall få eit testemne etter malen over.

Først må eg lese fila over alle som vert overført i natt frå FS:

In [12]:
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
svar = requests.get(url, headers=headers, params=params)
importerte_filar = svar.json()

no = datetime.now()
igår = (datetime.now() - timedelta(hours=24)).isoformat()
filliste = []
for i in importerte_filar['sis_imports']:
    import_dato = datetime.fromisoformat(i['created_at'].replace('Z', '+00:00')).replace(tzinfo=None)
    igår_dato = datetime.fromisoformat(igår)
    if (import_dato > igår_dato):
        for j in i['csv_attachments']:
            if j['filename'] == "users_filtered.csv":
                filliste.append({'dato': i['created_at'], 'url': j['url']})
users_liste = []
for i in filliste:
    temp = pd.read_csv(StringIO(requests.get(i['url'], headers=headers).content.decode("utf-8")))
    print(f"{i['dato']}: {len(temp)}")
    users_liste.append(temp)

overført = pd.concat(users_liste)
overført['brukarnamn'] = overført['login_id'].apply(lambda x: x.split('@')[0])
overført = overført[overført['brukarnamn'].str.isalpha()]
nye = overført[~overført['brukarnamn'].isin(eksisterande['brukarnamn'])]

2025-05-20T09:11:17Z: 1
2025-05-20T08:11:17Z: 1
2025-05-20T02:22:08Z: 18686
2025-05-19T13:11:14Z: 1
2025-05-19T12:11:15Z: 1


In [13]:
nye

,user_id,integration_id,login_id,first_name,last_name,short_name,email,status,brukarnamn
8754,fs:203:1386940,NaN,siq@hvl.no,Sundus,Iqbal,NaN,sundus.iqbal@hvl.no,active,siq
12558,fs:203:530754,NaN,stte@hvl.no,Ståle Bye,Teslo,NaN,stale.teslo@hvl.no,active,stte
15471,fs:203:821302,NaN,chme@hvl.no,Christian Kjellstrøm,Melby,NaN,christian.melby@hvl.no,active,chme
17129,fs:203:926199,NaN,arhk@hvl.no,Arve Henning,Kvilhaug,NaN,arve.henning.kvilhaug@hvl.no,active,arhk
0,fs:203:821302,NaN,chme@hvl.no,Christian Kjellstrøm,Melby,NaN,christian.melby@hvl.no,active,chme


Dersom eg ser at det er nye tilsette skal eg lage nye testemner for dei, og eg må legge dei til i fila `users_filtered.csv`: 

In [23]:
dato = datetime.now().strftime("%y%m%d")
data = pd.read_csv("users_filtered.csv")
n = 1
for row, index in nye.iterrows():
#     params = {
#         'course[name]': f"Testemne for {row['Namn']}",
#         'course[sis_course_id]': f"hvl{dato}-e{n}-akv",
#         'course[course_code]': f"Testemne for {row['Namn']}",
#         'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne.\nTing du lager her kan du hente inn i andre emne til bruk i undervising.",
#         'course[default_view]': 'syllabus', 
#     }
#     url = f"https://hvl.instructure.com/api/v1/accounts/9044/courses"
#     response = requests.post(url, headers=headers, params=params)
#     course_id = response.json()['id']
#     params = {
#         'enrollment[user_id]': i['user_id'],
#         'enrollment[type]': 'TeacherEnrollment',
#         'enrollment[workflow_state]': 'active',
#     }
#     n += 1
#     url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/enrollments"
#     response = requests.post(url, headers=headers, params=params)
#     print(f"{i['user_id']}: {response.status_code}")

    # Legg nye til i `users_filtered.csv`
    data.loc[len(data)] = [index[0], index[1], index[2], index[3], index[4], index[5], index[6], index[7]]

data.to_csv("users_filtered.csv", index=False)

user_id                fs:203:1386940
integration_id                    NaN
login_id                   siq@hvl.no
first_name                     Sundus
last_name                       Iqbal
short_name                        NaN
email             sundus.iqbal@hvl.no
status                         active
brukarnamn                        siq
Name: 8754, dtype: object
user_id                fs:203:530754
integration_id                   NaN
login_id                 stte@hvl.no
first_name                 Ståle Bye
last_name                      Teslo
short_name                       NaN
email             stale.teslo@hvl.no
status                        active
brukarnamn                      stte
Name: 12558, dtype: object
user_id                    fs:203:821302
integration_id                       NaN
login_id                     chme@hvl.no
first_name          Christian Kjellstrøm
last_name                          Melby
short_name                           NaN
email             christia

C:\Users\AKV\AppData\Local\Temp\ipykernel_4236\3763825849.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.loc[len(data)] = [index[0], index[1], index[2], index[3], index[4], index[5], index[6], index[7]]
C:\Users\AKV\AppData\Local\Temp\ipykernel_4236\3763825849.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.loc[len(data)] = [index[0], index[1], index[2], index[3], index[4], index[5], index[6], index[7]]
C:\Users\AKV\AppData\Local\Temp\ipykernel_4236\3763825849.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treate